### 2개의 층
은닉층을 포함해서 출력층까지의 개수<br>
다중 분류이기때문에 출력층에서 softmax 함수를 사용한다. - 이진 분류라면 sigmoid 함수를 사용.<br>
은닉층에도 활성 함수를 사용해한다. 굳이 사용하지 않으면 여러 개의 산술식을 합쳐놓은 것과 같은 것이라서 굳이 층을 나눌 필요가 없어짐.(산술함수를 사용하지 않으면 뒤에 있는 출력층과 합쳐서 하나의 층으로도 나타낼 수 있게 됨.)<br>
은닉층에서 많이 사용하는 것이 sigmoid(시그모이드), relu(렐루), tanh(하이퍼볼릭 탄젠트)...<br>이런 선형함수들은 케라스에 준비되어있음.<br><br>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 심층 신경망
은닉층 - hidden layer - **h**<br>
은닉층의 객체를 dense1으로 만들고 출력층의 객체를 dense2로 만들자.

In [2]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [3]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

이미지의 픽셀을 0~255에서 0~1으로 변환.<br>
28X28인 2차원 배열을 펼쳐서 1차원으로 만든다 → 784 1차원배열

In [4]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10,activation='softmax')

model = keras.Sequential([dense1,dense2])

은닉층은 sigmoid, 출력층은 softmax를 활성함수로 사용.<br>

케라스는 summary 메서드를 이용하여 모델의 구조를 출력할 수 있다.

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


Dense class 객체를 만들 때 name 매개변수로 층의 이름을 지정해주지 않았기때문에,  순서대로 dense는 dense1, dense_1은 dense2이다.<br><br>
 (None, 100) 
 - 100개의 뉴런이 있기때문에 100개의 출력이 만들어짐
 - 앞에 차원은 None. fit()을 출력할 때 batchsize를 지정할 수 있는데, 기본 값은 32이다. 미니배치 경사하강법을 사용한다. <br>
 훈련(fit)할 때 배치 크기를 바꿀 수 있다. 변경할 때마다 모델의 구성이 달라지면 안되기 때문에 모델을 만드는 과정은 하나로하고, 훈련할 때 매개변수로 바꿀 수 있도록 프레임워크를 설계해야한다.<br>
 케라스도 한 배치에 들어갈 수 있는 차원의 개수를 None으로 지정해놓고, 자연스럽게 fit 함수에서 바꿀 수 있도록..
 
Param : 78500개<br>
 - (입력층의 784개의 뉴런) X (은닉층의 100개의 뉴런) + (100개의 뉴런마다 있는 절편 100개)

### 층을 추가하는 다른 방법
#### 방법 1

In [6]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')
], name='패션 MNIST 모델')

따로 dense1, dense2 객체를 만들지 않고 그냥  Dense class를 객체화하자마자 바로 Sequential 클래스에 저장하는 방법.

In [7]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


#### 방법 2 - 실제로 많이 사용

In [8]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

Sequential 클래스 객체를 만들고, **add** 메소드를 이용하여 dense class 추가.<br>
add()의 장점은 add 메소드 중간에 if 문을 사용해서 프로그램 동적으로 사용 가능<br>(ex.층을 넣고 빼고...유연하게 사용 가능)

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [10]:
# 손실함수 지정, 정확도도 측정하도록 설정
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
# 모델 훈련. 5에포크 훈련한다.
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5621 - accuracy: 0.8100
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4070 - accuracy: 0.8527
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3727 - accuracy: 0.8654
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3499 - accuracy: 0.8744
Epoch 5/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3329 - accuracy: 0.8801


1에포크 결과  : 4s 2ms/step - loss: 0.5621 - accuracy: 0.8100<br>
5 에포크 결과 : 4s 2ms/step - loss: 0.3329 - accuracy: 0.8801<br>
에포크가 진행될수록 loss가 떨어지고, 정확도는 증가한다.<br>

In [11]:
# 성능 평가
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3495 - accuracy: 0.8727


[0.3495326042175293, 0.8726666569709778]

15_인공신경망과 동일한 compile 설정이다.<br>
이전에 85% 정확도였다면, 층이 추가되어 87.3% 정도의 정확도를 가지게 되었다.<br>
추가된 층이 성능을 향상시킴

### 렐루 활성화 함수와 Flatten 층

초기 인공신경망 은닉층에서는 활성화 함수로 sigmoid 함수가 많이 사용되었다.<br>
하지만 시그모이드 함수는 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있어 올바른 출력값이 나오지 않음 - 포화됐다. 층을 깊게 쌓기 힘들다.<br><br>
이를 개선하기위해 **렐루**가 나왔다.<br>

렐루 함수는 z가 0보다 크면 z를 출력하고, z가 음수일 경우 0을 출력한다.<br>

__ReLu 함수는 심층 신경망에서 뛰어나다 - 특히 이미지처리__
<br>

추가적으로, 기존에 28X28 2차원 배열을 인공 신경망에 주입하기위해 reshape을 사용했는데 케라스의 **Flatten** 클래스를 사용해도 된다.<br>
Flatten 클래스는 층처럼 입력층과 은닉층 사이에 추가하기 때문에 층이라고 부른다.(편의를 위해 만든 케라스에만 있는 유틸리티)

In [12]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


flatten 층의 param을 보면 0이다.<br>
실제로 의미있는 작업을 한건 아니고, 편의를 위해 존재하는 것.<br>
28X28 이미지를 1차원 배열로 펼치는 작업을 플래튼층이 해준다.<br>
플래튼 층의 summary 모델의 출력값만 봐도 이 신경망의 입력 값은 784개 1차원 배열이구나를 알 수 있다.

바뀐 것은 flatten층으로 펼친것과, sigmoid를 Relu로 바꿔준 것 뿐<br>
렐루 함수때문에 조금 더 성능이 좋아 질 것

In [14]:
# 훈련 데이터 다시 준비
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [15]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5289 - accuracy: 0.8122
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3927 - accuracy: 0.8596
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3540 - accuracy: 0.8730
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3330 - accuracy: 0.8809
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3190 - accuracy: 0.8858


2ms/step - loss: 0.3190 - accuracy: 0.8858<br>
훈련 데이터에서 정확도 아주 조금 증가

In [16]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3622 - accuracy: 0.8763


[0.362192839384079, 0.8763333559036255]

[0.362192839384079, 0.8763333559036255]<br>
검증 데이터 정확도 87.6%..아주 조금 더 좋아짐

### 옵티마이저

옵티마이저는 기본적으로 확률적 경사 하강법을 사용한다.<br>


In [17]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

컴파일 메소드에서 optimizer='sgd라고 쓰면 확률적 경사 하강법이다.<br>
하지만 샘플을 진짜 하나씩 뽑아서 사용하는건 아니고, 미니배치라고 생각하고 사용하는 것이다.

In [18]:
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')

tensorflow.keras.optimizers 아래 SGD 클래스로 구현되어있다.<br>
원래 이렇게 사용하는건데, 편의를 위해 optimizer='sgd'라고 쓰면 자동으로 SGD 클래스를 만들어준다.<br>
클래스의 매개변수값을 바꿔서 사용하고 싶다면 윗 코드와 같이 객체를 만들어서 사용해야한다.

In [19]:
sgd = keras.optimizers.SGD(learning_rate=0.1)

학습률을 바꾸고싶다면 learning_rate 매개변수를 사용한다.

In [20]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공한다.<br>
momentum은 모멘텀 최적화, nesterov는 네스테로프 모멘텀 최적화를 사용한다.

적응적 학습률을 사용하는 옵티마이저는 Adagrad, RMSprop 등이 있으며, <br>
모멘텀 최적화와 RMSprop의 장점을 접목한 것이 Adam이다.<br><br>
입문서이다보니 상세 이론은 설명하지 않는다.

In [21]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [22]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorical_crossentropy', metrics='accuracy')

In [23]:
# 모델 만들기
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
# 훈련
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5240 - accuracy: 0.8176
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3963 - accuracy: 0.8564
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3567 - accuracy: 0.8695
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3272 - accuracy: 0.8807
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3098 - accuracy: 0.8869


In [25]:
# 학습결과 평가
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3603 - accuracy: 0.8716


[0.36034518480300903, 0.8715833425521851]

| 앙상블  | optimizer |
|:---:|:---:|
| (Random forest) 여러 개의 모델을 가지고 앙상블 | (은닉층) 여러 개의 뉴런을 가지고 출력값을 만든다. |
| 각각의 개별Tree가 독립적으로 훈련.<br>tree를 여러개의 코어로 나누어서 훈련을 한 뒤 합침.<br>다른 트리와 관계없이 훈련 | 은닉층의 유닛들이 동시에 훈련된다.|
| 결과 위에 다른 층을 쌓을 수 없음 | 여러 개의 층을 쌓아서 각 층에서 훈련된 결과를 가지고 단계별로 학습 가능. |
| DB, 엑셀같은 정형화된 데이터에 적합 | 이미지, 텍스트 같은 비정형화된 데이터에 적합 |
